In [14]:
import numpy as np
import pandas as pd
import Levenshtein as L

from fuzzywuzzy import fuzz
from pprint import pprint

In [2]:
print(dir(fuzz))

['QRatio', 'SequenceMatcher', 'UQRatio', 'UWRatio', 'WRatio', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '_process_and_sort', '_token_set', '_token_sort', 'partial_ratio', 'partial_token_set_ratio', 'partial_token_sort_ratio', 'platform', 'ratio', 'token_set_ratio', 'token_sort_ratio', 'unicode_literals', 'utils', 'warnings']


In [3]:
data = pd.read_csv('digilib.tsv', sep='\t', index_col=0)
data = data.replace(np.nan, '')

In [88]:
data.tail()

,title,creator,contributor,nidn,date,keyword,description,fakultas,prodi
51861,PELACAKAN OBJEK DAN PENGAMBIL GAMBAR WAJAH PAD...,Rafi Dasa Nanda Putra,"Mervin Tangguar Hutabarat, Muhammad Shiddiq Sa...",0018056501;0023108802,2020-11-03 13:32:09,"Kriminalitas, Pencacahan, Pendeteksian, Pengen...",Tingginya angka kriminalitas di Indonesia teru...,STEI,Teknik Elektro
51862,METODE KETERLIBATAN STAKEHOLDER UNTUK IMPLEMEN...,Hesti Maheswari,"Gatot Yudoko, M.A.Sc., Akbar Adhi Utama",0025076102;0005067707,2020-11-20 14:48:01,"metode keterlibatan, sustainable reverse logis...",Pengolahan sampah elektronik di Indonesia dido...,SBM,Sains Manajemen
51863,EVALUASI TARIF PDAM KOTA BANDUNG BERDASARKAN P...,Raden Ajeng Koesoemo Roekmi,"Widiarto, MCRP.",,2020-11-21 16:18:38,"Prinsip Keterjangkauan, Prinsip Keadilan, Stan...",Penyediaan air bersih oleh Pemerintah Daerah h...,SAPPK,Perencanaan Wilayah dan Kota
51864,DAMPAK KINERJA USAHA MIKRO KECIL DAN MENENGAH ...,Faqih Ahmad Muzakky,Subiakto,0014046017,2020-11-24 13:36:51,"inklusi keuangan, UMKM, e-commerce",Sebuah pertanyaan telah banyak diangkat dalam ...,SBM,Sains Manajemen
51865,USULAN STRATEGI USAHA PERUSAHAAN KONSULTAN ITB...,Novi Fitriani,Deddy Priatmodjo Koesrindartoto,0003047006,2020-11-27 15:23:57,"Strategi Bisnis, Akuisisi, Peningkatan Keuanga...",Institut Teknologi Bandung (ITB) merupakan per...,,Magister Bisnis dan Administrasi


In [107]:
U = lambda x, y: L.setratio(x, y)

In [108]:
def elastic(query, many=5):
    
    total = pd.DataFrame()
    for on, searchkey in query.items():
        total[on] = data[on].apply(lambda string: U(string.split(),
                                                    searchkey.split()))
    
    result = sorted([(avg,index) for (index, avg)
                     in total.mean(axis=1).to_dict().items()],
                    reverse=True)[:many]
    
    return result

In [109]:
%%time

x = elastic({'description': 'gempa bumi',
             'description': 'machine learning'})
x

CPU times: user 14.7 s, sys: 19.9 ms, total: 14.7 s
Wall time: 14.8 s


[(0.4832535885167464, 37972),
 (0.43343653250773995, 23622),
 (0.4, 878),
 (0.3453947368421053, 6819),
 (0.3314285714285714, 24328)]

In [110]:
data.loc[[i for _,i in x]]

,title,creator,contributor,nidn,date,keyword,description,fakultas,prodi
37972,Mine ventilation,A. Skochinsky,"V. Komarov, John, Scott, Dmitri, Telyankovsky",,2019-04-30 16:02:38,Mining. Mine air. Mine dust. Ventilation.,Mine Ventilation,,lain
23622,A SIMULATED ANNEALING HEURISTIC FOR THE HYBRID...,Oktavianto Jimat Wibowo,Yosi Agustina Hidayat,,2017-09-27 14:50:41,"HYBRID VEHICLE ROUTING PROBLEM (HVRP), tenaga ...",kelestarian lingkungan,FTI,Teknik dan Manajemen Industri
878,DIFFUSION ATOMIQUE DANS UN RESEAU LOCAL A DIFF...,Mohamad Sukrisno Mardijanto,"Jury, 1. Jean Ferrie 2. Michele Cart 3. Claude...",,2004-12-07 09:56:56,reseau local diffusion non fiable,nothing abstract,,
6819,Keselamatan & Kesehatan Kerja Kinerja K3 di PC,Suseno Kramadibrata Dr,,,2007-04-17 15:20:10,Health & Safety,Significant Incidents,,
24328,PENGARUH SOLVENT TERHADAP DINAMIKA NONLINEAR M...,Siti Latifah,Freddy P.Zen,,2017-11-20 09:43:18,"Dinamika mikrotubulus model-U, solvent, kink-a...",Abstrak Sesuai lampiran,,


In [112]:
U(data.loc[37972].to_dict()['description'], 'gempa bumi')

0.3076923076923077

In [113]:
U(data.loc[37972].to_dict()['description'], 'machine learning')

0.625